In [ ]:
import os

from datasets import load_dataset
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm

In [ ]:
def bbox_intersect(bbox1, bbox2):
    x1, y1, x2, y2 = bbox1
    x3, y3, x4, y4 = bbox2
    return x2 > x3 and x4 > x1 and y2 > y3 and y4 > y1

In [ ]:
def divide_bboxes(bbox1, bbox2):
    x1, y1, x2, y2 = bbox1
    x3, y3, x4, y4 = bbox2

    divided_boxes = []
    if x3 > x1:
        divided_boxes.append([x1, y1, x3, y2])  # Left part
    if x4 < x2:
        divided_boxes.append([x4, y1, x2, y2])  # Right part
    if y3 > y1:
        divided_boxes.append([x1, y1, x2, y3])  # Top part
    if y4 < y2:
        divided_boxes.append([x1, y4, x2, y2])  #Bottom part
    return divided_boxes

In [ ]:
def area(bbox):
    x1, y1, x2, y2 = bbox
    return (x2 - x1) * (y2 - y1)

In [ ]:
# global background_bbox
# background_bbox = [0, 0, 0, 0]
def get_background_bboxes(background_bbox, handled_bbox, bboxes):
    if len(bboxes) != 0:
        bbox = bboxes[0]
        divided_bboxes = divide_bboxes(handled_bbox, bbox)
        for divided_bbox in divided_bboxes:
            if area(divided_bbox) > area(background_bbox):
                get_background_bboxes(background_bbox, divided_bbox, [bb for bb in bboxes if bbox_intersect(divided_bbox, bb)])
    else:
        if area(handled_bbox) > area(background_bbox):
            background_bbox[:] = handled_bbox

In [ ]:
dataset = load_dataset("HichTala/dota")

In [ ]:
sample = dataset['train'][42738]
image = sample['image']
image_id = sample['image_id']
bboxes = sample['objects']['bbox']
bbox_ids = sample["objects"]["bbox_id"]
labels = sample["objects"]["category"]
sample['image_id']

In [ ]:
bboxes

In [ ]:
dataset

In [ ]:
bboxes

In [ ]:
new_format = []
for bbox in bboxes:
    x1, y1, w, h = bbox
    x2, y2 = w + x1, h + y1
    new_format.append((x1, y1, x2, y2))
bboxes = new_format

In [ ]:
global background_bbox
background_bbox = [0, 0, 0, 0]
get_background_bboxes(background_bbox, [0, 0, image.size[0], image.size[1]], bboxes)
background_bbox

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 8))
ax.imshow(image)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 8))
ax.imshow(image)

# Add bounding boxes
for bbox, label in zip(bboxes, labels):
    # x1, y1, x2, y2 = bbox
    x1, y1, w, h = bbox
    # w, h = x2 - x1, y2 - y1
    rect = patches.Rectangle((x1, y1), w, h, linewidth=2,
                             edgecolor="red", facecolor="none")
    # ax.add_patch(rect)
    # ax.text(x1, y1, label, color="white", fontsize=10,
    #         bbox=dict(facecolor="red", alpha=0.5))

# x1, y1, x2, y2 = background_bbox
# w, h = x2 - x1, y2 - y1
# rect = patches.Rectangle((x1, y1), w, h, linewidth=2,
#                          edgecolor="green", facecolor="none")
# ax.add_patch(rect)

# plt.axis("off")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 8))
ax.imshow(image)

# Add bounding boxes
for bbox, label in zip(bboxes, labels):
    # x1, y1, x2, y2 = bbox
    x1, y1, w, h = bbox
    # w, h = x2 - x1, y2 - y1
    rect = patches.Rectangle((x1, y1), w, h, linewidth=2,
                             edgecolor="red", facecolor="none")
    # ax.add_patch(rect)
    # ax.text(x1, y1, label, color="white", fontsize=10,
    #         bbox=dict(facecolor="red", alpha=0.5))

# x1, y1, x2, y2 = background_bbox
# w, h = x2 - x1, y2 - y1
# rect = patches.Rectangle((x1, y1), w, h, linewidth=2,
#                          edgecolor="green", facecolor="none")
# ax.add_patch(rect)

# plt.axis("off")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 8))
ax.imshow(image)

# Add bounding boxes
for bbox, label in zip(bboxes, labels):
    # x1, y1, x2, y2 = bbox
    x1, y1, w, h = bbox
    # w, h = x2 - x1, y2 - y1
    rect = patches.Rectangle((x1, y1), w, h, linewidth=2,
                             edgecolor="red", facecolor="none")
    # ax.add_patch(rect)
    # ax.text(x1, y1, label, color="white", fontsize=10,
    #         bbox=dict(facecolor="red", alpha=0.5))

# x1, y1, x2, y2 = background_bbox
# w, h = x2 - x1, y2 - y1
# rect = patches.Rectangle((x1, y1), w, h, linewidth=2,
#                          edgecolor="green", facecolor="none")
# ax.add_patch(rect)

# plt.axis("off")
plt.show()

In [ ]:
image.crop((x1, y1, x2, y2)).size

In [ ]:
# for bbox, bbox_id, label in zip(bboxes, bbox_ids, labels):
#     x1, y1, x2, y2 = bbox
#     w, h = x2 - x1, y2 - y1
#     image.crop((x1, y1, x2, y2)).save(f"{image_id}_{bbox_id}.png")

In [ ]:
# labels = ['background'] + dataset["train"].features["objects"]['category'].feature.names
# label2id, id2label = {}, {}
# for i, label in enumerate(labels):
#     label2id[label] = str(i)
#     id2label[str(i)] = label

In [ ]:
# for sample in tqdm(dataset['train']):
#     image = sample['image']
#     image_id = sample['image_id']
#     bboxes = sample['objects']['bbox']
#     bbox_ids = sample["objects"]["bbox_id"]
#     labels = sample["objects"]["category"]
#
#     if image_id == 200365:
#         bboxes.pop(2)
#         labels.pop(2)
#         bbox_ids.pop(2)
#     if image_id == 550395:
#         bboxes.pop()
#         labels.pop()
#         bbox_ids.pop()
#
#     global background_bbox
#     background_bbox = [0, 0, 0, 0]
#     get_background_bboxes(background_bbox, [0, 0, image.size[0], image.size[1]], bboxes)
#     w_min, h_min = image.size
#
#     for bbox, bbox_id, label in zip(bboxes, bbox_ids, labels):
#         x1, y1, x2, y2 = bbox
#         w, h = x2 - x1, y2 - y1
#         w_min = min(w_min, w)
#         h_min = min(h_min, h)
#         os.makedirs(f"dataset/train/{id2label[str(label+1)]}", exist_ok=True)
#         image.crop((x1, y1, x2, y2)).save(f"dataset/train/{id2label[str(label+1)]}/{image_id}_{bbox_id}.png")
#
#     x1, y1, x2, y2 = background_bbox
#     os.makedirs(f"dataset/train/{id2label['0']}", exist_ok=True)
#     w, h = x2 - x1, y2 - y1
#     if w > w_min/2 and h > h_min/2 and area(background_bbox) > 25:
#         image.crop((x1, y1, x2, y2)).save(f"dataset/train/{id2label['0']}/{image_id}_background.png")

In [ ]:
dataset = load_dataset("imagefolder", data_dir="dataset", drop_labels=False)

In [ ]:
dataset = load_dataset(type="ImageFolder", path="dataset")


In [ ]:
dataset.push_to_hub("HichTala/dota-background")

In [ ]:
for sample in tqdm(dataset['train']):
    image = sample['image']
    image_id = sample['image_id']
    bboxes = sample['objects']['bbox']
    bbox_ids = sample["objects"]["bbox_id"]
    labels = sample["objects"]["category"]

    if image_id == 200365:
        bboxes.pop(2)
        labels.pop(2)
        bbox_ids.pop(2)
    if image_id == 550395:
        bboxes.pop()
        labels.pop()
        bbox_ids.pop()

    global background_bbox
    background_bbox = [0, 0, 0, 0]
    get_background_bboxes(background_bbox, [0, 0, image.size[0], image.size[1]], bboxes)
    w_min, h_min = image.size

    for bbox, bbox_id, label in zip(bboxes, bbox_ids, labels):
        x1, y1, x2, y2 = bbox
        w, h = x2 - x1, y2 - y1
        w_min = min(w_min, w)
        h_min = min(h_min, h)
        os.makedirs(f"dataset/train/{id2label[str(label+1)]}", exist_ok=True)
        image.crop((x1, y1, x2, y2)).save(f"dataset/train/{id2label[str(label+1)]}/{image_id}_{bbox_id}.png")

    x1, y1, x2, y2 = background_bbox
    os.makedirs(f"dataset/train/{id2label['0']}", exist_ok=True)
    w, h = x2 - x1, y2 - y1
    if w > w_min and h > h_min and area(background_bbox) > 25:
        image.crop((x1, y1, x2, y2)).save(f"dataset/train/{id2label['0']}/{image_id}_background.png")

In [ ]:
dataset = load_dataset("HichTala/coco-background")

In [ ]:
exclude_idx = [17725, 85356, 140667, 243422, 569617, 838562, 838563]

In [ ]:
len(dataset['train'])

In [ ]:
dataset['train'] = dataset['train'].select(
    (
        i for i in range(len(dataset['train']))
        if i not in set(exclude_idx)
    )
)

In [ ]:
for sample in tqdm(dataset['train']):
    image = sample['image']

In [ ]:
dataset['train'][838564]

In [ ]:
dataset.push_to_hub("HichTala/coco-background")

In [ ]:
from datasets import load_dataset
# Full dataset
dataset = load_dataset("HuggingFaceM4/COCO")
print(dataset)

In [ ]:
# Training set only
dataset = load_dataset("rafaelpadilla/coco2017", split="train")

# Validation set only
dataset = load_dataset("rafaelpadilla/coco2017", split="val")

In [ ]:
for sample in tqdm(dataset['train']):
    bboxes = sample['objects']['bbox']
    image_id = sample['image_id']

    if image_id == 200365:
        bboxes.pop(2)
        labels.pop(2)
        bbox_ids.pop(2)
    if image_id == 550395:
        bboxes.pop()
        labels.pop()
        bbox_ids.pop()

    for bbox in bboxes:
        x1, y1, x2, y2 = bbox
        w, h = x2 - x1, y2 - y1
        bbox = [x1, y1, w, h]

In [ ]:
sample = new_dataset['train'][36912]
image = sample['image']
image_id = sample['image_id']
bboxes = sample['objects']['bbox']
bbox_ids = sample["objects"]["bbox_id"]
labels = sample["objects"]["category"]
sample['image_id']

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 8))
ax.imshow(image)

# Add bounding boxes
for bbox, label in zip(bboxes, labels):
    # x1, y1, x2, y2 = bbox
    x1, y1, w, h = bbox
    # w, h = x2 - x1, y2 - y1
    rect = patches.Rectangle((x1, y1), w, h, linewidth=2,
                             edgecolor="red", facecolor="none")
    ax.add_patch(rect)
    ax.text(x1, y1, label, color="white", fontsize=10,
            bbox=dict(facecolor="red", alpha=0.5))
# ax.add_patch(rect)

# plt.axis("off")
plt.show()

In [ ]:
def change_format(sample):
    image_id = sample['image_id']
    bboxes = sample['objects']['bbox']
    labels = sample["objects"]["category"]
    bbox_ids = sample["objects"]["bbox_id"]

    if image_id == 200365:
        bboxes.pop(2)
        labels.pop(2)
        bbox_ids.pop(2)
    if image_id == 550395:
        bboxes.pop()
        labels.pop()
        bbox_ids.pop()

    new_format = []
    for bbox in bboxes:
        x1, y1, x2, y2 = bbox
        w, h = x2 - x1, y2 - y1
        new_format.append((x1, y1, w, h))
    sample['objects']['bbox'] = new_format
    sample['objects']['category'] = labels
    sample['objects']['bbox_id'] = bbox_ids

    return sample

In [ ]:
dataset = load_dataset("detection-datasets/coco")

In [ ]:
sample = dataset['train'][36912]
new_sample = change_format(sample)

In [ ]:
new_sample

In [ ]:
sample

In [ ]:
new_dataset = dataset.map(change_format)

In [ ]:
new_dataset.push_to_hub("HichTala/coco")